# Aplicacion Codebar con Flask

In [ ]:
# librerias

from flask import Flask, render_template, Response, request, Markup

import cv2
from pyzbar.pyzbar import decode
from PIL import Image
import pandas as pd
import ast
import pymongo
import requests
import matplotlib.pyplot as plt
%matplotlib inline


import warnings
warnings.simplefilter('ignore')

In [ ]:
app=Flask(__name__)
cam=cv2.VideoCapture(0)

In [ ]:
def captura():
    while 1:
        ret, frame=cam.read()
        if ret==False: break   
        frame=cv2.imencode('.jpg', frame)[1].tobytes()
        yield (b'--frame\r\n'b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [ ]:
@app.route('/', methods=['POST', 'GET'])
def main():
    return render_template('index.html')

In [ ]:
@app.route('/video')
def video():
    return Response(captura(), mimetype='multipart/x-mixed-replace; boundary=frame')

In [ ]:
@app.route('/tomar_foto', methods = ['POST'])
def tomar_foto():
    name = request.form['name']
    print(name)
    _, frame = video.read()
    cv2.imwrite(f'capturas/{name}.jpg', frame)
    return Response(status = 200)

In [ ]:
if __name__=='__main__':
    app.run(debug=False)